In [ ]:
pip install beautifulsoup4

In [ ]:
pip install requests

In [50]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

def products_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # print(soup.prettify())

    # Targeting the <ul> element with id "product-grid"
    product_grid = soup.find('ul', id='product-grid')

    # Targeting the <li> elements within the <ul> element
    li_elements = product_grid.find_all('li')

    # Extracting the links from the <li> elements and joining with the base URL
    base_url = 'https://watchcollectors.co.uk'  # Replace with the actual base URL
    all_product = [urljoin(base_url, li.find('a')['href']) for li in li_elements if li.find('a')]

    return all_product

In [51]:
imgUrls=[];productName=[];Price=[];modelNumber=[];modelYear=[];Gender=[];Diameter=[]
originalBox=[];originalPaper=[];productUrls=[]

def Product_details(links):
    c = 0
    for i in links:

        response = requests.get(i)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        try:
            # image_urls = [img['data-src'] for img in soup.select('.media img[data-src]')]
            image_urls = [img['data-src'] for img in soup.select('.media img[data-src]')]
            # Create clickable links and store them in a list
            # clickable_links = []
            # for url in image_urls:
            #     clickable_link = f'<a href="{url}" target="_blank"><img src="{url}" alt="Product Image"></a>'
            #     imgUrls.append(clickable_link)
            # clickable_links = ['<a href="{}" target="_blank"><img src="{}" alt="Product Image"></a>'.format(url, url) for url in image_urls]
            imgUrls.append(image_urls)
        except:
            imgUrls.append('')

        productUrls.append(i)
        
        try:
            product_name = soup.select_one('.card-information__text a').get_text(strip=True)
            productName.append(product_name)
        except:
            productName.append('')
            
        try:
            price = soup.select_one('.price-item--sale').get_text(strip=True)
            Price.append(price)
        except:
            Price.append('')
        
        try:
            
            # Find the <td> element with the text "Model Number"
            model_number_td = soup.find('td', string='Model Number')
            # If the <td> element is found, extract the model number from the following <td>
            if model_number_td:
                model_number = model_number_td.find_next('td').get_text(strip=True)
            modelNumber.append(model_number)
        except:
            modelNumber.append('')
    
        try:
            year_td = soup.find('td', string='Year')
            if yr:
                model_year = soup.select_one('.model-year-class').get_text(strip=True)  # Update with the actual class or selector
            modelYear.append(model_year )
        except:
            modelYear.append('')
    
        try:
            gender = soup.select_one('.gender-class').get_text(strip=True)  # Update with the actual class or selector
            Gender.append(gender)
        except:
            Gender.append('')

        try:
            diameter = soup.select_one('.diameter-class').get_text(strip=True)  # Update with the actual class or selector
            Diameter.append(diameter)
        except:
            Diameter.append('')

        try:
            original_box = soup.select_one('.original-box-class').get_text(strip=True)  # Update with the actual class or selector
            originalBox.append(original_box)
        except:
            originalBox.append('')
        
        try:
            original_papers = soup.select_one('.original-papers-class').get_text(strip=True)  # Update with the actual class or selector
            originalPaper.append(original_papers)
        except:
            originalPaper.append('')
        
        break
        
    #     c = c + 1
    #     print(f"Product {c} completed.")
    
    # with open( file_name + '.csv', 'w', encoding='utf-8', newline='') as file:
    #     writer = csv.writer(file)
    #     writer.writerow(['Title','Title Url', 'Product Number', 'Short Description', 'Price', 'Image Url', 'Description'])
        
    #     for i in range(len(Title)):
    #         writer.writerow([Title[i],Title_url[i], Product_number[i], Short_des[i], Price[i], Image_url[i], Description[i]])
    
    return True

In [52]:
url = input("Enter Url: ")
all_links = products_links(url)

# Print the list of links
print(all_links)


['https://watchcollectors.co.uk/products/choaprd', 'https://watchcollectors.co.uk/products/louis-vuitton-ref-tambour-xl-minute-repeater-travel-time-18k-rose-gold-box-papers', 'https://watchcollectors.co.uk/products/rolex-lady-datejust-pearlmaster-ref-69298-white-mother-of-pearl-diamond-dial-18k-yellow-gold', 'https://watchcollectors.co.uk/products/rolex-yacht-master-40-ref-126622-slate-dial-box-papers-2021-stainless-steel-platinum', 'https://watchcollectors.co.uk/products/audemars-piguet-royal-oak-chronograph-ref-26603st-oo-d002cr-01000-blue-dial-box-papers-2010-stainless-steel', 'https://watchcollectors.co.uk/products/rolex-daytona-ref-116520-black-dial-2012-box-papers-stainless-steel-1', 'https://watchcollectors.co.uk/products/rolex-daytona-beach-blue-ref-116509-18k-white-gold-2007', 'https://watchcollectors.co.uk/products/vacheron-constantin-malte-chronograph-ref-49180-champagne-dial-40mm-18k-rose-gold', 'https://watchcollectors.co.uk/products/cartier-tank-americaine-chronograph-ref

In [53]:
Product_details(all_links)

True

In [55]:
print(imgUrls)
print(productUrls)
print(productName)
print(Price)
print(modelNumber)
print(modelYear)
print(Gender)
print(Diameter)
print(originalBox)
print(originalPaper)

[['//watchcollectors.co.uk/cdn/shop/files/Chopard18KRoseGoldSecondHandWatchCollectors1_533x.jpg?v=1702574924', '//watchcollectors.co.uk/cdn/shop/files/Chopard18KRoseGoldSecondHandWatchCollectors2_533x.jpg?v=1702574927', '//watchcollectors.co.uk/cdn/shop/files/LouisVuitton18KRoseGoldSecondhandWatchCollectors1_533x.jpg?v=1702471203', '//watchcollectors.co.uk/cdn/shop/files/LouisVuitton18KRoseGoldSecondhandWatchCollectors2_533x.jpg?v=1702571602', '//watchcollectors.co.uk/cdn/shop/files/Rolex18KYellowGoldPearlmastersecondHandWatchcollectors1_533x.jpg?v=1702471209', '//watchcollectors.co.uk/cdn/shop/files/Rolex18KYellowGoldPearlmastersecondHandWatchcollectors2_533x.jpg?v=1702471209', '//watchcollectors.co.uk/cdn/shop/products/AP_01_533x.jpg?v=1652702498', '//watchcollectors.co.uk/cdn/shop/products/AP_02_533x.jpg?v=1652702498']]
['https://watchcollectors.co.uk/products/choaprd']
['Chopard L.U.C 150 ALL-IN-ONE | REF. 161925-5001 | Limited to 15 Pieces | Tourbillon, Perpetual Calendar, Equatio